# 영화 추천 시스템
## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [7]:
# 사용하지 않는 컬럼 제거
ratings = ratings.drop(columns=['timestamp', 'genre'])
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2. 데이터 분석

In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [10]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화 추가

In [13]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('truman', regex=False)]

,movie_id,title,genre
1636,1682,"Truman Show, The (1998)",Drama


In [14]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [2571, 1721, 2028, 593, 50]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'dongho'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['dongho']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,dongho,2571,5,"Matrix, The (1999)"
1,dongho,1721,5,Titanic (1997)
2,dongho,2028,5,Saving Private Ryan (1998)
3,dongho,593,5,"Silence of the Lambs, The (1991)"
4,dongho,50,5,"Usual Suspects, The (1995)"


In [15]:

if not ratings.isin({'user_id':['dongho']})['user_id'].any():  # user_id에 'dongho'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,dongho,2571,5,"Matrix, The (1999)"
836479,dongho,1721,5,Titanic (1997)
836480,dongho,2028,5,Saving Private Ryan (1998)
836481,dongho,593,5,"Silence of the Lambs, The (1991)"
836482,dongho,50,5,"Usual Suspects, The (1995)"


## 4. indexing 후 CSR Matrix 생성

In [16]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [17]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,2571,5,124
836479,6039,1721,5,27
836480,6039,2028,5,48
836481,6039,593,5,121


In [18]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련시키기

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [23]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
dongho, titanic = user_to_idx['dongho'], movie_to_idx['Titanic (1997)']
dongho_vector, titanic_vector = als_model.user_factors[dongho], als_model.item_factors[titanic]

In [24]:
dongho_vector

array([-4.19675261e-01,  9.76156771e-01,  9.95015800e-01, -3.38103443e-01,
        9.07823518e-02, -5.68957865e-01, -3.77297282e-01,  1.77017123e-01,
        5.82970917e-01,  4.72127408e-01, -2.37797186e-01, -1.06939726e-01,
        1.78382069e-01, -5.51368475e-01, -7.40169168e-01,  4.83207911e-01,
       -5.36431745e-02, -2.45441839e-01, -2.30636045e-01, -5.86522967e-02,
        4.20273215e-01, -5.87044358e-01,  6.46164417e-01,  2.56208237e-02,
        5.19966304e-01,  1.04439028e-01, -3.79240036e-01, -5.83084822e-01,
       -2.79170781e-01, -3.85925919e-01, -9.07606035e-02,  9.27326083e-01,
        1.39446700e+00, -5.96596897e-01,  2.99442440e-01,  6.34488642e-01,
        2.94268131e-01,  1.30149648e-01, -2.23030016e-01,  2.13430285e-01,
       -4.72055107e-01, -7.07198679e-01,  9.01562095e-01, -2.78673708e-01,
       -5.86967021e-02, -6.42159343e-01,  2.87026972e-01,  4.31782454e-01,
       -7.90466666e-01,  8.09915125e-01,  9.48521942e-02, -4.56880599e-01,
        4.33791727e-01, -

In [25]:
titanic_vector

array([-1.3065977e-02,  3.0114315e-02,  3.7587482e-02, -1.8288529e-03,
       -2.2467444e-02, -1.6129812e-03, -1.3964948e-02,  2.6309572e-02,
       -7.5523453e-03,  5.0450690e-02,  3.9827861e-03,  1.6253410e-02,
       -1.3126511e-02, -8.2904946e-05,  2.4149143e-03,  7.2678416e-03,
        3.3844836e-02,  1.1777965e-02,  2.8263807e-02, -9.7278822e-03,
        1.6361756e-02, -1.4950836e-02,  3.2546427e-02,  1.3327436e-03,
       -1.6058151e-02, -4.5159385e-03, -1.6811254e-02, -1.6030222e-02,
       -1.3782299e-02,  1.9413098e-03,  8.1026182e-03,  4.2903699e-02,
        1.0513170e-02, -1.9435000e-02, -1.6760169e-02, -4.4387574e-03,
       -2.7078961e-04,  3.3564303e-02,  4.1354410e-02,  2.4836129e-02,
       -1.1615628e-02, -2.3940548e-02,  1.4187363e-02,  1.8908972e-02,
        2.2610754e-02,  1.4829210e-02, -4.2803241e-03,  1.1503653e-02,
        5.3628283e-03,  2.4009554e-02, -1.3444178e-03, -2.5178881e-02,
        3.0111039e-02, -2.3341238e-02,  2.4238270e-02, -2.3584036e-02,
      

In [26]:
# 나와 타이타닉의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(dongho_vector, titanic_vector)

0.5425123

In [27]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
matrix = movie_to_idx['Men in Black (1997)']
matrix_vector = als_model.item_factors[matrix]
np.dot(dongho_vector, matrix_vector)

0.12108431

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [28]:
favorite_movie = 'Truman Show, The (1998)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(385, 1.0),
 (641, 0.66016644),
 (465, 0.65639687),
 (39, 0.6416586),
 (386, 0.6183099),
 (248, 0.6046016),
 (406, 0.6000329),
 (533, 0.5983315),
 (843, 0.5652895),
 (429, 0.5638141),
 (384, 0.544161),
 (437, 0.53180456),
 (483, 0.50907433),
 (18, 0.50703263),
 (1679, 0.48995358)]

In [29]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Truman Show, The (1998)',
 "Mr. Holland's Opus (1995)",
 'Quiz Show (1994)',
 'Apollo 13 (1995)',
 'Philadelphia (1993)',
 'Good Will Hunting (1997)',
 'People vs. Larry Flynt, The (1996)',
 'Fried Green Tomatoes (1991)',
 'Primary Colors (1998)',
 'Scent of a Woman (1992)',
 'Jerry Maguire (1996)',
 "What's Eating Gilbert Grape (1993)",
 'Trainspotting (1996)',
 'Awakenings (1990)',
 'Pump Up the Volume (1990)']

In [30]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [31]:
get_similar_movie('Good Will Hunting (1997)')

['Good Will Hunting (1997)',
 'Apollo 13 (1995)',
 'Shawshank Redemption, The (1994)',
 'Truman Show, The (1998)',
 'Dead Man Walking (1995)',
 'Jerry Maguire (1996)',
 'Awakenings (1990)',
 'Pulp Fiction (1994)',
 'River Runs Through It, A (1992)',
 'Quiz Show (1994)']

In [32]:
get_similar_movie('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Face/Off (1997)',
 'Men in Black (1997)',
 'Twelve Monkeys (1995)']

## 8. 내가 가장 좋아할만한 영화 추천

In [33]:
user = user_to_idx['dongho']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(23, 0.6241458),
 (157, 0.57955444),
 (38, 0.53474724),
 (51, 0.47298563),
 (222, 0.43543792),
 (224, 0.40661967),
 (141, 0.3835776),
 (92, 0.38245773),
 (269, 0.3232935),
 (87, 0.31348288),
 (99, 0.31083056),
 (248, 0.29429197),
 (44, 0.28768417),
 (220, 0.27435914),
 (289, 0.24091966),
 (472, 0.23810177),
 (67, 0.23170897),
 (41, 0.23151286),
 (107, 0.22962447),
 (117, 0.21546242)]

In [34]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

["Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Sixth Sense, The (1999)',
 'Fargo (1996)',
 'Pulp Fiction (1994)',
 'L.A. Confidential (1997)',
 'Fugitive, The (1993)',
 'Terminator 2: Judgment Day (1991)',
 'GoodFellas (1990)',
 'Braveheart (1995)',
 'American Beauty (1999)',
 'Good Will Hunting (1997)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Seven (Se7en) (1995)',
 'Reservoir Dogs (1992)',
 'Sling Blade (1996)',
 'Gladiator (2000)',
 'Rain Man (1988)',
 'Jurassic Park (1993)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)']

In [35]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Schindler\'s List (1993)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Saving Private Ryan (1998)', 0.2729785373866175),
 ('Silence of the Lambs, The (1991)', 0.23850259577248606),
 ('Usual Suspects, The (1995)', 0.06979226708820838),
 ('Titanic (1997)', 0.050079320047121165),
 ('Matrix, The (1999)', -0.015472096437276412)]